In [10]:
from langchain_community.chat_models import ChatOllama
from langchain_core.prompts import ChatPromptTemplate
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_community.embeddings import OllamaEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain.document_loaders import TextLoader  # Untuk file .txt
from langchain.document_loaders import PyPDFLoader  # Untuk file .pdf
import os

# 1. Fungsi untuk Mengunggah dan Membaca Dokumen
def load_document(file_path):
    if file_path.endswith(".txt"):
        loader = TextLoader(file_path)
    elif file_path.endswith(".pdf"):
        loader = PyPDFLoader(file_path)
    else:
        raise ValueError("Format file tidak didukung. Gunakan file .txt atau .pdf.")
    documents = loader.load()
    return documents

# 2. Fungsi untuk Split Dokumen menjadi Chunks
def split_document(documents):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
    texts = text_splitter.split_documents(documents)
    return texts

# 3. Fungsi untuk Membuat Vector Store
def create_vector_store(texts):
    embeddings = OllamaEmbeddings(model="llama3.2:1b-instruct-fp16")
    vectorstore = FAISS.from_documents(texts, embeddings)
    return vectorstore

# 4. Fungsi untuk Meringkas Dokumen
def summarize_document(vectorstore, query):
    llm = ChatOllama(model="llama3.2:1b-instruct-fp16", temperature=0)
    prompt_template = ChatPromptTemplate.from_template(
        "Buat ringkasan lengkap dan terstruktur dari teks berikut. Ringkasan harus mencakup semua poin penting dan disajikan dengan rapi:\n\n{context}\n\nRingkasan:"
    )
    retriever = vectorstore.as_retriever(search_kwargs={"k": 5})  # Ambil lebih banyak chunk untuk ringkasan lengkap
    rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | prompt_template
        | llm
    )
    response = rag_chain.invoke(query)
    return response.content

# Path ke file PDF atau TXT
file_path = "artikel1.pdf"  # Ganti dengan path file Anda

# Memuat dokumen
try:
    print("Memuat dokumen...")
    documents = load_document(file_path)
    print("Dokumen berhasil dimuat.")
except Exception as e:
    print(f"Error saat membaca file: {str(e)}")
    raise

# Membagi dokumen menjadi chunks
print("Membagi dokumen menjadi chunks...")
texts = split_document(documents)

# Membuat vector store
print("Membuat vector store...")
vectorstore = create_vector_store(texts)

# Meringkas dokumen
print("Meringkas dokumen...")
query = "Buat ringkasan lengkap dan terstruktur dari dokumen ini."
summary = summarize_document(vectorstore, query)

# Tampilkan hasil
print("\nRingkasan Dokumen:")
print(summary)

Memuat dokumen...
Dokumen berhasil dimuat.
Membagi dokumen menjadi chunks...
Membuat vector store...
Meringkas dokumen...

Ringkasan Dokumen:
Berikut adalah ringkasan lengkap dan terstruktur dari teks yang Anda berikan:

**Teks**

Merupakan langkah yang digunakan untuk pengujian sistem penjualan barang di Swalayan Brastagi Medan. Dalam pre-procesing data, salah satu langkah yang dilakukan adalah transformasi setiap nilai atribut yang sama ke bentuk numerik sehingga mudah dilakukan untuk proses pemecahan masalah dan bentukan data dan sampelnya.

**Pre-Procesing Data**

1. Nama barang: nama barang yang akan dijadikan acuan untuk dimasukan kedalam daftar stok barang yang dibutuhkan.
2. Penjualan pada Swalayan Brastagi Medan: Journal of Computing and Informatics Research, Vol 1, No 1, November 2021.

**Pola Belanja Pembeli**

Sistem yang berjalan pada Swalayan Brastagi Medan dalam hal Pola Belanja Pembeli menggunakan sebuah sistem pencatatan dengan menggunakan media komputer. Dimana pola b

RAG Advanced (Multi File)